In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentences = ["i like dog", "i love coffe", "i hate milk", "you like cat", "you love milk",
            "you hate coffe"]
dtype = torch.float

In [3]:
word_list = list(set(" ".join(sentences).split()))
print(word_list)
# " ".join(sentences)로 모든 문장들을 띄어쓰기로 구분하여 합침
# set은 중복을 허용하지 않음을 이용하여 word를 모두 각자 저장할 수 있게 됨

['love', 'like', 'coffe', 'milk', 'i', 'you', 'dog', 'hate', 'cat']


In [4]:
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)
print(word_dict)

{'love': 0, 'like': 1, 'coffe': 2, 'milk': 3, 'i': 4, 'you': 5, 'dog': 6, 'hate': 7, 'cat': 8}


In [5]:
import torch.utils.data as data

input_batch = []
target_batch = []
batch_size = len(sentences)


def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
        target_batch.append(target)
    
    return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)        

In [6]:
print(input_batch)
print(len(word_dict))
print()
print(target_batch)

tensor([[[0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]]], requires_grad=True)
9

tensor([6, 2, 3, 8, 3, 2])


In [7]:
n_step = 2 # 학습하고자 하는 문장 길이 - 1
n_hidden = 5 # 은닉층 사이즈

class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
        self.Softmax = nn.Softmax(dim=1)
        
    def forward(self, hidden, X):
        X = X.transpose(0, 1)
        outputs, hidden = self.rnn(X, hidden)
        outputs = outputs[-1]
        model = torch.mm(outputs, self.W) + self.b
        return model

In [8]:
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

/usr/local/Caskroom/miniforge/base/envs/pytorch/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
    output = model(hidden, input_batch)
    loss = criterion(output, target_batch)

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0100 cost = 0.465591
Epoch: 0200 cost = 0.082999
Epoch: 0300 cost = 0.033771
Epoch: 0400 cost = 0.019468
Epoch: 0500 cost = 0.012973


In [10]:
input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences])
print('->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']]
-> ['dog', 'coffe', 'milk', 'cat', 'milk', 'coffe']
